In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import h3
from scipy.sparse import lil_matrix
from torch_geometric.utils import from_scipy_sparse_matrix
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader, Dataset


In [2]:
# Debug Helper Function
def debug(message, variable=None):
    print(f"[DEBUG] {message}")
    if variable is not None:
        print(variable)


# Load the data
data = pd.read_csv('dataset/sales_4.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Select top N stores
top_n = 100
store_lifetime_product_count = data.groupby('store_id')['product_count'].sum()
top_n_stores = store_lifetime_product_count.sort_values(ascending=False).head(top_n).index
filtered_data = data[data['store_id'].isin(top_n_stores)]

debug("Filtered data for top n stores", filtered_data.head())
debug("Shape of filtered data", filtered_data.shape)



[DEBUG] Filtered data for top n stores
        date locality_type product_name  product_count   latitude   longitude  \
1 2021-01-01       Diamond         Beta              1  40.858416  -73.781928   
2 2021-01-01       Diamond        Alpha              1  33.363745 -118.424787   
4 2021-01-01       Diamond        Alpha              1  30.769735  -91.458505   
5 2021-01-01       Diamond        Gamma              1  41.274267  -81.993373   
6 2021-01-01       Diamond        Delta              1  29.967208  -97.319137   

   store_id  
1         2  
2         3  
4         5  
5         6  
6         7  
[DEBUG] Shape of filtered data
(6401132, 7)


In [3]:
# Aggregate data to weekly demand per store
filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_data = filtered_data.groupby(['store_id', 'week']).agg({
    'product_count': 'sum',
    'latitude': 'first',
    'longitude': 'first',
    'locality_type': 'first'
}).reset_index()

debug("Shape of weekly_data after aggregation", weekly_data.shape)


C:\Users\rznis\AppData\Local\Temp\ipykernel_13160\1361998663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['week'] = filtered_data['date'].dt.to_period('W').apply(lambda r: r.start_time)


[DEBUG] Shape of weekly_data after aggregation
(20448, 6)


In [4]:
# Normalize demand
weekly_data['product_count'] = (weekly_data['product_count'] - weekly_data['product_count'].mean()) / weekly_data['product_count'].std()
locality_mapping = {'Diamond': 0, 'Gold': 1, 'Silver': 2}
weekly_data['locality_type'] = weekly_data['locality_type'].map(locality_mapping)

debug("First few rows after encoding locality_type", weekly_data.head())

time_series = weekly_data.pivot(index='store_id', columns='week', values='product_count').fillna(0)
debug("Shape of time_series", time_series.shape)



[DEBUG] First few rows after encoding locality_type
   store_id       week  product_count   latitude  longitude  locality_type
0         2 2020-12-28      -0.916058  40.858416 -73.781928              0
1         2 2021-01-04       3.803672  40.858416 -73.781928              0
2         2 2021-01-11       4.115463  40.858416 -73.781928              0
3         2 2021-01-18       3.834851  40.858416 -73.781928              0
4         2 2021-01-25       3.760801  40.858416 -73.781928              0
[DEBUG] Shape of time_series
(100, 205)


In [5]:

# Create adjacency matrix using H3 indexing
resolution = 3
weekly_data['h3_index'] = weekly_data.apply(
    lambda row: h3.latlng_to_cell(row['latitude'], row['longitude'], resolution), axis=1
)
h3_to_store_map = weekly_data.groupby('h3_index')['store_id'].apply(list).to_dict()
h3_neighbors = {h: h3.grid_disk(h, 1) for h in h3_to_store_map.keys()}

num_stores = len(weekly_data['store_id'].unique())
adj_matrix = lil_matrix((num_stores, num_stores))
store_to_idx = {store_id: idx for idx, store_id in enumerate(weekly_data['store_id'].unique())}

for h3_index, neighbors in h3_neighbors.items():
    stores_in_hex = h3_to_store_map.get(h3_index, [])
    for neighbor in neighbors:
        neighbor_stores = h3_to_store_map.get(neighbor, [])
        for s1 in stores_in_hex:
            for s2 in neighbor_stores:
                idx1 = store_to_idx[s1]
                idx2 = store_to_idx[s2]
                adj_matrix[idx1, idx2] = 1

adj_sparse = adj_matrix.tocsr()
edge_index, edge_weight = from_scipy_sparse_matrix(adj_sparse)
debug("Shape of edge_index", edge_index.shape)
debug("Shape of edge_weight", edge_weight.shape)


[DEBUG] Shape of edge_index
torch.Size([2, 294])
[DEBUG] Shape of edge_weight
torch.Size([294])


In [6]:


# Prepare node features
locality_encoded = weekly_data.groupby('store_id')['locality_type'].first()
locality_encoded = pd.get_dummies(locality_encoded, prefix='locality_type')
locality_tensor = torch.tensor(locality_encoded.values, dtype=torch.float)
node_features = torch.tensor(time_series.values, dtype=torch.float)
node_features = torch.cat([node_features, locality_tensor], dim=1)

debug("Updated node features shape", node_features.shape)



[DEBUG] Updated node features shape
torch.Size([100, 208])


In [9]:
# Define the STGNN model class
class STGNN(nn.Module):
    def __init__(self, in_channels, spatial_out, temporal_out, forecast_steps):
        super(STGNN, self).__init__()
        self.gcn = GCNConv(in_channels, spatial_out)
        self.temporal_conv = nn.Conv1d(spatial_out, temporal_out, kernel_size=3, padding=1)
        self.fc = nn.Linear(temporal_out, forecast_steps)

    def forward(self, x, edge_index, edge_weight):
        batch_size, num_nodes, time_steps = x.shape
        x = x.permute(0, 2, 1).reshape(-1, num_nodes)  # Correct reshaping for GCNConv
        debug("Shape after reshaping for GCN", x.shape)

        # Spatial convolution
        x = self.gcn(x, edge_index, edge_weight)
        x = torch.relu(x)

        # Reshape for temporal convolution
        x = x.view(batch_size, num_nodes, -1).permute(0, 2, 1)
        debug("Shape before temporal convolution", x.shape)

        # Temporal convolution
        x = self.temporal_conv(x)
        x = torch.relu(x)

        # Flatten for final prediction
        x = x.mean(dim=-1)
        x = self.fc(x)
        debug("Shape after fully connected layer", x.shape)
        return x





# Dataset Class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, time_steps, forecast_steps):
        self.data = data
        self.time_steps = time_steps
        self.forecast_steps = forecast_steps

    def __len__(self):
        return self.data.shape[1] - self.time_steps - self.forecast_steps

    def __getitem__(self, idx):
        x = self.data[:, idx:idx + self.time_steps]
        y = self.data[:, idx + self.time_steps:idx + self.time_steps + self.forecast_steps]
        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)


# Training setup
time_steps = 8
forecast_steps = 1
spatial_out = 32
temporal_out = 64
epochs = 10
batch_size = 16
learning_rate = 0.001

dataset = TimeSeriesDataset(node_features, time_steps, forecast_steps)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


model = STGNN(
    in_channels=node_features.shape[1],  # Ensure this matches node feature dimension
    spatial_out=spatial_out,
    temporal_out=temporal_out,
    forecast_steps=forecast_steps
)



criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()
        
        # Reshape input for STGNN
        x = x.permute(0, 2, 1)  # (batch_size, num_nodes, time_steps)
        
        # Forward pass
        out = model(x, edge_index, edge_weight)
        
        # Adjust target shape
        y = y.mean(dim=-1)  # Match prediction dimensions
        
        # Compute loss
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")



[DEBUG] Shape after reshaping for GCN
torch.Size([1600, 8])


C:\Users\rznis\AppData\Local\Temp\ipykernel_13160\2838479059.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1600x8 and 208x32)

In [ ]:
# Forecast
model.eval()
with torch.no_grad():
    x = node_features[:, -time_steps:].unsqueeze(0).permute(0, 2, 1)  # Single batch
    prediction = model(x, edge_index, edge_weight)
    print("Prediction:", prediction)
